In [37]:
import matplotlib.pyplot as plt
import numpy as np

# Plotting only enabled for elems_per_side <= 4
elems_per_side = 16
comp_elems_per_side =  elems_per_side # // 2 + 1
num_ranks = 4
elems_per_plane = elems_per_side * comp_elems_per_side
elems_per_domain = elems_per_plane * elems_per_side
planes_per_rank = elems_per_side // num_ranks

# Create 3d array with numbers
numbers_3d = np.arange(elems_per_domain).reshape(elems_per_side, elems_per_side, comp_elems_per_side)
print("Original 3D array shape:", numbers_3d.shape)
print("Total elements:", numbers_3d.size)
print("Value range:", numbers_3d.min(), "to", numbers_3d.max())

def plot_3d_array_subplot(arr, ax, title="3D Array Visualization"):
    """Plot a 3d array in 3D space with grid lines on given axis"""
    # Create coordinate grids
    y, z, x = np.meshgrid(range(arr.shape[0]), range(arr.shape[1]), range(arr.shape[2]), indexing='xy')
    
    # Flatten everything for plotting
    x_flat = x.flatten()
    y_flat = y.flatten()
    z_flat = z.flatten()
    numbers_flat = arr.flatten()
    
    # Use 16 colors cycling
    colors = numbers_flat // elems_per_side
    
    # Plot the scattered points
    scatter = ax.scatter(x_flat, y_flat, z_flat, c=colors, cmap='tab20', s=80, alpha=0.9)
    
    # Add grid lines
    # Lines along X direction (keeping Y,Z constant)
    for j in range(arr.shape[1]):  # Y direction
        for k in range(arr.shape[2]):  # Z direction
            x_line = range(arr.shape[0])
            y_line = [j] * arr.shape[0]
            z_line = [k] * arr.shape[0]
            ax.plot(x_line, y_line, z_line, 'k-', alpha=0.3, linewidth=0.5)
    
    # Lines along Y direction (keeping X,Z constant)
    for i in range(arr.shape[0]):  # X direction
        for k in range(arr.shape[2]):  # Z direction
            x_line = [i] * arr.shape[1]
            y_line = range(arr.shape[1])
            z_line = [k] * arr.shape[1]
            ax.plot(x_line, y_line, z_line, 'k-', alpha=0.3, linewidth=0.5)
    
    # Lines along Z direction (keeping X,Y constant)
    for i in range(arr.shape[0]):  # X direction
        for j in range(arr.shape[1]):  # Y direction
            x_line = [i] * arr.shape[2]
            y_line = [j] * arr.shape[2]
            z_line = range(arr.shape[2])
            ax.plot(x_line, y_line, z_line, 'k-', alpha=0.3, linewidth=0.5)
    
    # Add number labels
    for xi, yi, zi, num in zip(x_flat, y_flat, z_flat, numbers_flat):
        ax.text(xi, yi, zi, str(num), fontsize=7)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(title)
    
    # Set axis limits to show full range
    ax.set_xlim(0, 3)
    ax.set_ylim(0, 3)
    ax.set_zlim(0, 3)
    
    return scatter


print("CYCLIC TRANSITIONS - X → Z → Y → X:")
print("="*50)

if elems_per_side <= 4:
    # Create figure with 2x2 subplot layout
    fig = plt.figure(figsize=(12, 8))

# Step 1: Original (X contiguous)
print("Step 1: Original array (X contiguous)")

if elems_per_side <= 4:
    ax1 = fig.add_subplot(221, projection='3d')
    scatter1 = plot_3d_array_subplot(numbers_3d, ax1, "Step 1: Original (X contiguous)")

# Step 2: X → Z (make Z contiguous)
# This means: (Z,Y,X) → (Y,X,Z)
step2 = np.transpose(numbers_3d, (1, 2, 0)) 
print("\nStep 2: X→Z transition (Z contiguous)")
print(f"Transpose: (Z,Y,X) → (Y,X,Z)")
    
if elems_per_side <= 4:
    ax2 = fig.add_subplot(222, projection='3d')
    scatter2 = plot_3d_array_subplot(step2, ax2, "Step 2: X→Z (Z contiguous)")

# Step 3: Z → Y (make Y contiguous) 
# This means: (Y,X,Z) → (Z,X,Y)
step3 = np.transpose(step2, (2, 1, 0))
print("\nStep 3: Z→Y transition (Y contiguous)")
print(f"Transpose: (Y,X,Z) → (Z,X,Y)")
    
if elems_per_side <= 4:
    ax3 = fig.add_subplot(223, projection='3d')
    scatter3 = plot_3d_array_subplot(step3, ax3, "Step 3: Z→Y (Y contiguous)")

# Step 4: Y → X (back to X contiguous, should match original)
# This means: (Z,X,Y) → (Z,Y,X)
step4 = np.transpose(step3, (0, 2, 1))
print("\nStep 4: Y→X transition (back to X contiguous)")
print(f"Transpose: (Z,X,Y) → (Z,Y,X)")
    
if elems_per_side <= 4:
    ax4 = fig.add_subplot(224, projection='3d')
    scatter4 = plot_3d_array_subplot(step4, ax4, "Step 4: Y→X (back to X contiguous)")

# Verify we're back to the original
print(f"\nVerification: Arrays are identical: {np.array_equal(numbers_3d, step4)}")

if elems_per_side <= 4:
    plt.tight_layout()
    plt.show()

Original 3D array shape: (16, 16, 16)
Total elements: 4096
Value range: 0 to 4095
CYCLIC TRANSITIONS - X → Z → Y → X:
Step 1: Original array (X contiguous)

Step 2: X→Z transition (Z contiguous)
Transpose: (Z,Y,X) → (Y,X,Z)

Step 3: Z→Y transition (Y contiguous)
Transpose: (Y,X,Z) → (Z,X,Y)

Step 4: Y→X transition (back to X contiguous)
Transpose: (Z,X,Y) → (Z,Y,X)

Verification: Arrays are identical: True


In [38]:
# Verify split and concatenation results in same array
split_arrays = []
for i in range(num_ranks):
    slice_array = numbers_3d[i*planes_per_rank:i*planes_per_rank+planes_per_rank, :, :]
    split_arrays.append(slice_array)

# Unite them back using concatenate
united_arrays = np.concatenate(split_arrays, axis=0)

print(f"\nVerification: Arrays are identical: {np.array_equal(numbers_3d, united_arrays)}")


Verification: Arrays are identical: True


In [39]:
# Simulate real transposition across ranks

# Prepare send buffer
shape = split_arrays[0].shape
print(f"Initial shape of split array: {shape}")
print(f"Initial rank[0] array: \n{split_arrays[0]}")

send_buffer = np.zeros(elems_per_domain).reshape(num_ranks, elems_per_side, comp_elems_per_side, planes_per_rank)
    
shape = send_buffer[0].shape
print(f"\nShape of rank[0] send buffer: {shape}")

# Fill send buffer with z-contiguous pre communication data
# In z-direction only [planes_per_rank] elements can be contiguous because the other values are on a different rank.
# This organization enables sending in batches
for rank in range(num_ranks):
    for idx_p in range(planes_per_rank):
        for idx_comp in range(comp_elems_per_side):
            for idx_num in range(elems_per_side):
                send_buffer[rank][idx_num, idx_comp, idx_p] = split_arrays[rank][idx_p, idx_num, idx_comp]
                
print(f"Rank[0] send buffer: \n{send_buffer[0]}")

Initial shape of split array: (4, 16, 16)
Initial rank[0] array: 
[[[   0    1    2 ...   13   14   15]
  [  16   17   18 ...   29   30   31]
  [  32   33   34 ...   45   46   47]
  ...
  [ 208  209  210 ...  221  222  223]
  [ 224  225  226 ...  237  238  239]
  [ 240  241  242 ...  253  254  255]]

 [[ 256  257  258 ...  269  270  271]
  [ 272  273  274 ...  285  286  287]
  [ 288  289  290 ...  301  302  303]
  ...
  [ 464  465  466 ...  477  478  479]
  [ 480  481  482 ...  493  494  495]
  [ 496  497  498 ...  509  510  511]]

 [[ 512  513  514 ...  525  526  527]
  [ 528  529  530 ...  541  542  543]
  [ 544  545  546 ...  557  558  559]
  ...
  [ 720  721  722 ...  733  734  735]
  [ 736  737  738 ...  749  750  751]
  [ 752  753  754 ...  765  766  767]]

 [[ 768  769  770 ...  781  782  783]
  [ 784  785  786 ...  797  798  799]
  [ 800  801  802 ...  813  814  815]
  ...
  [ 976  977  978 ...  989  990  991]
  [ 992  993  994 ... 1005 1006 1007]
  [1008 1009 1010 ... 1021 102

In [40]:
# Simulate all to all communication
recv_buffer = np.zeros_like(send_buffer)

for src_rank in range(num_ranks):
    for dst_rank in range(num_ranks):
        src_slice = slice(dst_rank * planes_per_rank, (dst_rank + 1) * planes_per_rank)
        dst_slice = slice(src_rank * planes_per_rank, (src_rank + 1) * planes_per_rank)
        recv_buffer[dst_rank][dst_slice] = send_buffer[src_rank][src_slice]
        
print(f"Rank[0] receive buffer: \n{recv_buffer[0]}")

Rank[0] receive buffer: 
[[[0.000e+00 2.560e+02 5.120e+02 7.680e+02]
  [1.000e+00 2.570e+02 5.130e+02 7.690e+02]
  [2.000e+00 2.580e+02 5.140e+02 7.700e+02]
  ...
  [1.300e+01 2.690e+02 5.250e+02 7.810e+02]
  [1.400e+01 2.700e+02 5.260e+02 7.820e+02]
  [1.500e+01 2.710e+02 5.270e+02 7.830e+02]]

 [[1.600e+01 2.720e+02 5.280e+02 7.840e+02]
  [1.700e+01 2.730e+02 5.290e+02 7.850e+02]
  [1.800e+01 2.740e+02 5.300e+02 7.860e+02]
  ...
  [2.900e+01 2.850e+02 5.410e+02 7.970e+02]
  [3.000e+01 2.860e+02 5.420e+02 7.980e+02]
  [3.100e+01 2.870e+02 5.430e+02 7.990e+02]]

 [[3.200e+01 2.880e+02 5.440e+02 8.000e+02]
  [3.300e+01 2.890e+02 5.450e+02 8.010e+02]
  [3.400e+01 2.900e+02 5.460e+02 8.020e+02]
  ...
  [4.500e+01 3.010e+02 5.570e+02 8.130e+02]
  [4.600e+01 3.020e+02 5.580e+02 8.140e+02]
  [4.700e+01 3.030e+02 5.590e+02 8.150e+02]]

 ...

 [[3.088e+03 3.344e+03 3.600e+03 3.856e+03]
  [3.089e+03 3.345e+03 3.601e+03 3.857e+03]
  [3.090e+03 3.346e+03 3.602e+03 3.858e+03]
  ...
  [3.101e+03 3.

In [41]:
# Reorganize receive buffer for (1, 2, 0)

shape = recv_buffer[0].shape
print(f"Initial shape of recveive buffer: {shape}")

final_buffer_1_2_0 = np.zeros(elems_per_domain).reshape(num_ranks, planes_per_rank, comp_elems_per_side, elems_per_side)
    
shape = final_buffer_1_2_0[0].shape
print(f"\nShape of rank[0] final buffer: {shape}")

# Reorganize data into a contiguous layout for (1, 2, 0)
for rank in range(num_ranks):
    for idx_p in range(planes_per_rank):
        for idx_comp in range(comp_elems_per_side):
            for idx_num in range(elems_per_side):
                dst_idx_x = idx_p + (idx_num // planes_per_rank) * planes_per_rank
                dst_idx_z = idx_num % planes_per_rank
                final_buffer_1_2_0[rank][dst_idx_z, idx_comp, dst_idx_x] = recv_buffer[rank][idx_num, idx_comp, idx_p]
                
print("\nTransposition for (1, 2, 0) complete")
print(f"Rank[0] final buffer (1, 2, 0): \n{final_buffer_1_2_0[0]}")

Initial shape of recveive buffer: (16, 16, 4)

Shape of rank[0] final buffer: (4, 16, 16)

Transposition for (1, 2, 0) complete
Rank[0] final buffer (1, 2, 0): 
[[[0.000e+00 2.560e+02 5.120e+02 ... 3.328e+03 3.584e+03 3.840e+03]
  [1.000e+00 2.570e+02 5.130e+02 ... 3.329e+03 3.585e+03 3.841e+03]
  [2.000e+00 2.580e+02 5.140e+02 ... 3.330e+03 3.586e+03 3.842e+03]
  ...
  [1.300e+01 2.690e+02 5.250e+02 ... 3.341e+03 3.597e+03 3.853e+03]
  [1.400e+01 2.700e+02 5.260e+02 ... 3.342e+03 3.598e+03 3.854e+03]
  [1.500e+01 2.710e+02 5.270e+02 ... 3.343e+03 3.599e+03 3.855e+03]]

 [[1.600e+01 2.720e+02 5.280e+02 ... 3.344e+03 3.600e+03 3.856e+03]
  [1.700e+01 2.730e+02 5.290e+02 ... 3.345e+03 3.601e+03 3.857e+03]
  [1.800e+01 2.740e+02 5.300e+02 ... 3.346e+03 3.602e+03 3.858e+03]
  ...
  [2.900e+01 2.850e+02 5.410e+02 ... 3.357e+03 3.613e+03 3.869e+03]
  [3.000e+01 2.860e+02 5.420e+02 ... 3.358e+03 3.614e+03 3.870e+03]
  [3.100e+01 2.870e+02 5.430e+02 ... 3.359e+03 3.615e+03 3.871e+03]]

 [[3.20

In [42]:
# Transpose (2, 1, 0)

# Prepare send buffer
shape = recv_buffer[0].shape
# print(f"Initial shape: {shape}")
# print(f"Initial rank[0] array: \n{final_buffer_1_2_0[0]}")

send_buffer = np.zeros(elems_per_domain).reshape(num_ranks, elems_per_side, comp_elems_per_side, planes_per_rank)
final_buffer_2_1_0 = np.zeros_like(final_buffer_1_2_0)
    
shape = final_buffer_2_1_0[0].shape
print(f"\nShape of final buffer (2,1,0): {shape}")

# Prepare send buffer
for rank in range(num_ranks):
    for idx_p in range(planes_per_rank):
        for idx_comp in range(comp_elems_per_side):
            for idx_num in range(elems_per_side):
                send_buffer[rank][idx_num, idx_comp, idx_p] = final_buffer_1_2_0[rank][idx_p, idx_comp, idx_num]
                
# print(f"\nRank[0] send buffer: \n{send_buffer[0]}")

# Simulate all to all communication
for src_rank in range(num_ranks):
    for dst_rank in range(num_ranks):
        src_slice = slice(dst_rank * planes_per_rank, (dst_rank + 1) * planes_per_rank)
        dst_slice = slice(src_rank * planes_per_rank, (src_rank + 1) * planes_per_rank)
        recv_buffer[dst_rank][dst_slice] = send_buffer[src_rank][src_slice]
        
# print(f"\nRank[0] receive buffer: \n{recv_buffer[0]}")

# Reorganize into contiguous layout
for rank in range(num_ranks):
    for idx_p in range(planes_per_rank):
        for idx_comp in range(comp_elems_per_side):
            for idx_num in range(elems_per_side):
                dst_idx_x = idx_p + (idx_num // planes_per_rank) * planes_per_rank
                dst_idx_z = idx_num % planes_per_rank
                final_buffer_2_1_0[rank][dst_idx_z, idx_comp, dst_idx_x] = recv_buffer[rank][idx_num, idx_comp, idx_p]
                
print("\nTransposition for (2, 1, 0) complete")
print(f"Rank[0] final buffer (2, 1, 0): \n{final_buffer_2_1_0[0]}")


Shape of final buffer (2,1,0): (4, 16, 16)

Transposition for (2, 1, 0) complete
Rank[0] final buffer (2, 1, 0): 
[[[0.000e+00 1.600e+01 3.200e+01 ... 2.080e+02 2.240e+02 2.400e+02]
  [1.000e+00 1.700e+01 3.300e+01 ... 2.090e+02 2.250e+02 2.410e+02]
  [2.000e+00 1.800e+01 3.400e+01 ... 2.100e+02 2.260e+02 2.420e+02]
  ...
  [1.300e+01 2.900e+01 4.500e+01 ... 2.210e+02 2.370e+02 2.530e+02]
  [1.400e+01 3.000e+01 4.600e+01 ... 2.220e+02 2.380e+02 2.540e+02]
  [1.500e+01 3.100e+01 4.700e+01 ... 2.230e+02 2.390e+02 2.550e+02]]

 [[2.560e+02 2.720e+02 2.880e+02 ... 4.640e+02 4.800e+02 4.960e+02]
  [2.570e+02 2.730e+02 2.890e+02 ... 4.650e+02 4.810e+02 4.970e+02]
  [2.580e+02 2.740e+02 2.900e+02 ... 4.660e+02 4.820e+02 4.980e+02]
  ...
  [2.690e+02 2.850e+02 3.010e+02 ... 4.770e+02 4.930e+02 5.090e+02]
  [2.700e+02 2.860e+02 3.020e+02 ... 4.780e+02 4.940e+02 5.100e+02]
  [2.710e+02 2.870e+02 3.030e+02 ... 4.790e+02 4.950e+02 5.110e+02]]

 [[5.120e+02 5.280e+02 5.440e+02 ... 7.200e+02 7.360e

In [43]:
# (0, 2, 1) does not require cross memory communication

final_buffer_0_2_1 = np.zeros_like(split_arrays)
shape = final_buffer_0_2_1[0].shape
print(f"\nShape of final buffer (0,2,1): {shape}")

for rank in range(num_ranks):
    for idx_p in range(planes_per_rank):
        for idx_comp in range(comp_elems_per_side):
            for idx_num in range(elems_per_side):
                final_buffer_0_2_1[rank][idx_p, idx_num, idx_comp] = final_buffer_2_1_0[rank][idx_p, idx_comp, idx_num]
                
print("\nTransposition for (0, 2, 1) complete")
print(f"Rank[0] final buffer (0, 2, 1): \n{final_buffer_0_2_1[0]}")


Shape of final buffer (0,2,1): (4, 16, 16)

Transposition for (0, 2, 1) complete
Rank[0] final buffer (0, 2, 1): 
[[[   0    1    2 ...   13   14   15]
  [  16   17   18 ...   29   30   31]
  [  32   33   34 ...   45   46   47]
  ...
  [ 208  209  210 ...  221  222  223]
  [ 224  225  226 ...  237  238  239]
  [ 240  241  242 ...  253  254  255]]

 [[ 256  257  258 ...  269  270  271]
  [ 272  273  274 ...  285  286  287]
  [ 288  289  290 ...  301  302  303]
  ...
  [ 464  465  466 ...  477  478  479]
  [ 480  481  482 ...  493  494  495]
  [ 496  497  498 ...  509  510  511]]

 [[ 512  513  514 ...  525  526  527]
  [ 528  529  530 ...  541  542  543]
  [ 544  545  546 ...  557  558  559]
  ...
  [ 720  721  722 ...  733  734  735]
  [ 736  737  738 ...  749  750  751]
  [ 752  753  754 ...  765  766  767]]

 [[ 768  769  770 ...  781  782  783]
  [ 784  785  786 ...  797  798  799]
  [ 800  801  802 ...  813  814  815]
  ...
  [ 976  977  978 ...  989  990  991]
  [ 992  993  994 .

In [44]:
# Now as functions

def prepare_send_buffer_1_2_0(input_buffer):
    send_buffer = np.zeros(elems_per_domain).reshape(num_ranks, elems_per_side, comp_elems_per_side, planes_per_rank)
        
    for rank in range(num_ranks):
        for idx_p in range(planes_per_rank):
            for idx_comp in range(comp_elems_per_side):
                for idx_num in range(elems_per_side):
                    send_buffer[rank][idx_num, idx_comp, idx_p] = input_buffer[rank][idx_p, idx_num, idx_comp]
                
    return send_buffer


def prepare_send_buffer_2_1_0(input_buffer):
    send_buffer = np.zeros(elems_per_domain).reshape(num_ranks, elems_per_side, comp_elems_per_side, planes_per_rank)

    for rank in range(num_ranks):
        for idx_p in range(planes_per_rank):
            for idx_comp in range(comp_elems_per_side):
                for idx_num in range(elems_per_side):
                    send_buffer[rank][idx_num, idx_comp, idx_p] = input_buffer[rank][idx_p, idx_comp, idx_num]
                
    return send_buffer


def communicate_all_to_all(send_buffer):
    recv_buffer = np.zeros(elems_per_domain).reshape(num_ranks, elems_per_side, comp_elems_per_side, planes_per_rank)

    for src_rank in range(num_ranks):
        for dst_rank in range(num_ranks):
            src_slice = slice(dst_rank * planes_per_rank, (dst_rank + 1) * planes_per_rank)
            dst_slice = slice(src_rank * planes_per_rank, (src_rank + 1) * planes_per_rank)
            recv_buffer[dst_rank][dst_slice] = send_buffer[src_rank][src_slice]
        
    return recv_buffer


def reorganize_recv(recv_buffer):
    final_buffer = np.zeros(elems_per_domain).reshape(num_ranks, planes_per_rank, comp_elems_per_side, elems_per_side)
        
    for rank in range(num_ranks):
        for idx_p in range(planes_per_rank):
            for idx_comp in range(comp_elems_per_side):
                for idx_num in range(elems_per_side):
                    dst_idx_x = idx_p + (idx_num // planes_per_rank) * planes_per_rank
                    dst_idx_z = idx_num % planes_per_rank
                    final_buffer[rank][dst_idx_z, idx_comp, dst_idx_x] = recv_buffer[rank][idx_num, idx_comp, idx_p]
                
    return final_buffer


def real_dist_transposition_1_2_0(input_buffer):
    send_buffer = prepare_send_buffer_1_2_0(input_buffer)
    recv_buffer = communicate_all_to_all(send_buffer)
    final_buffer = reorganize_recv(recv_buffer)
    
    return final_buffer


def real_dist_transposition_2_1_0(input_buffer):
    send_buffer = prepare_send_buffer_2_1_0(input_buffer)
    recv_buffer = communicate_all_to_all(send_buffer)
    final_buffer = reorganize_recv(recv_buffer)
    
    return final_buffer


def real_dist_transposition_0_2_1(input_buffer):
    final_buffer = np.zeros(elems_per_domain).reshape(num_ranks, planes_per_rank, elems_per_side, comp_elems_per_side)

    for rank in range(num_ranks):
        for idx_p in range(planes_per_rank):
            for idx_comp in range(comp_elems_per_side):
                for idx_num in range(elems_per_side):
                    final_buffer[rank][idx_p, idx_num, idx_comp] = input_buffer[rank][idx_p, idx_comp, idx_num]
    
    return final_buffer

In [45]:
# Verify real distributed transposition
# Non complex regular:
comp_elems_per_side =  elems_per_side
numbers_3d = np.arange(elems_per_domain).reshape(elems_per_side, elems_per_side, comp_elems_per_side)

print("Regular non complex data")

# Step 1: X → Z (make Z contiguous)
step1 = np.transpose(numbers_3d, (1, 2, 0)) 

# Step 2: Z → Y (make Y contiguous) 
step2 = np.transpose(step1, (2, 1, 0))

# Step 3: Y → X (back to X contiguous, should match original)
step3 = np.transpose(step2, (0, 2, 1))

# Verify we're back to the original
print(f"\nVerification: Arrays are same as initial after transposes: {np.array_equal(numbers_3d, step3)}")

# Non complex distributed
split_arrays = []
for i in range(num_ranks):
    slice_array = numbers_3d[i*planes_per_rank:i*planes_per_rank+planes_per_rank, :, :]
    split_arrays.append(slice_array)
    
print("Distributed non complex data")

# Step 1: X → Z (make Z contiguous)
split_step1 = real_dist_transposition_1_2_0(split_arrays)
step1_dist = np.concatenate(split_step1, axis=0)
print(f"Verification Step 1 distributed: Arrays are identical: {np.array_equal(step1, step1_dist)}")

# Step 2: Z → Y (make Y contiguous) 
split_step2 = real_dist_transposition_2_1_0(split_step1)
step2_dist = np.concatenate(split_step2, axis=0)
print(f"Verification Step 2 distributed: Arrays are identical: {np.array_equal(step2, step2_dist)}")

# Step 3: Y → X (back to X contiguous, should match original)
split_step3 = real_dist_transposition_0_2_1(split_step2)
step3_dist = np.concatenate(split_step3, axis=0)
print(f"Verification Step 3 distributed: Arrays are identical: {np.array_equal(step3, step3_dist)}")

print("\n" + 50 * "=" + "\n")

# Complex:
comp_elems_per_side =  elems_per_side // 2 + 1
elems_per_plane = elems_per_side * comp_elems_per_side
elems_per_domain = elems_per_plane * elems_per_side
numbers_3d = np.arange(elems_per_domain).reshape(elems_per_side, elems_per_side, comp_elems_per_side)

print("Complex data")

# Step 1: X → Z (make Z contiguous)
step1 = np.transpose(numbers_3d, (1, 2, 0)) 

# Step 2: Z → Y (make Y contiguous) 
step2 = np.transpose(step1, (2, 1, 0))

# Step 3: Y → X (back to X contiguous, should match original)
step3 = np.transpose(step2, (0, 2, 1))

# Verify we're back to the original
print(f"\nVerification: Arrays are same as initial after transposes: {np.array_equal(numbers_3d, step3)}")

# Non complex distributed
split_arrays = []
for i in range(num_ranks):
    slice_array = numbers_3d[i*planes_per_rank:i*planes_per_rank+planes_per_rank, :, :]
    split_arrays.append(slice_array)
    
print("Distributed complex data")

# Step 1: X → Z (make Z contiguous)
split_step1 = real_dist_transposition_1_2_0(split_arrays)
step1_dist = np.concatenate(split_step1, axis=0)
print(f"Verification Step 1 distributed: Arrays are identical: {np.array_equal(step1, step1_dist)}")

# Step 2: Z → Y (make Y contiguous) 
split_step2 = real_dist_transposition_2_1_0(split_step1)
step2_dist = np.concatenate(split_step2, axis=0)
print(f"Verification Step 2 distributed: Arrays are identical: {np.array_equal(step2, step2_dist)}")

# Step 3: Y → X (back to X contiguous, should match original)
split_step3 = real_dist_transposition_0_2_1(split_step2)
step3_dist = np.concatenate(split_step3, axis=0)
print(f"Verification Step 3 distributed: Arrays are identical: {np.array_equal(step3, step3_dist)}")

Regular non complex data

Verification: Arrays are same as initial after transposes: True
Distributed non complex data
Verification Step 1 distributed: Arrays are identical: True
Verification Step 2 distributed: Arrays are identical: True
Verification Step 3 distributed: Arrays are identical: True


Complex data

Verification: Arrays are same as initial after transposes: True
Distributed complex data
Verification Step 1 distributed: Arrays are identical: True
Verification Step 2 distributed: Arrays are identical: True
Verification Step 3 distributed: Arrays are identical: True
